# Ingestion from S3/R2/etc example

## Important Note

You'll need to have a bucket, set either the AWS_PROFILE (for getting credentials from ~/.aws/credentials) or set the environment variables for the ACCESS_KEY and KEY_ID

If you are using actual AWS s3, or another s3 compatible service, you can set the 4 below as appropriate. (Note that for S3, you'll need to set the ACCESS/SECRET or the profile, but may want ENDPOINT/REGION set to None.

## About this

Kamiwaza will store credentials when you ingest; if you see the markdown in cell 4 you'll see an example of passing None explicitly, which tells Kamiwaza roughly "These credentials already exist in the catalog, so use them, and associate the metadata with the ingested items, but I'm not passing them now"; it is effectively credential re-use. While Community Edition has open access to credentials in the catalog, in Enterprise the pattern here is useful because someone with the credentials can store them, provide permission to use them to an end user via Kamiwaza, but the end user cannot **retrieve** the credentials.

You can pass `save_secrets=False` to the ingester classes to instruct them to use the secrets but **not** save them in the secret store. (which means, of course, you'll have to supply them to retrieve)

## About this notebook

We've placed this outside the core notebooks specifically because you need to set up a bucket. You can upload the folder created in the 01 notebook as the sample parquet, however. Use the correct BUCKET_NAME and BUCKET_PATH for your upload, as per the top cell.

In [ ]:
# Constants you may need to set for this to run

BUCKET_NAME='kamiwaza-public'

BUCKET_PATH='testdata/'

# You only need to set this if you are letting the code read from your ~/.aws/credentials file
AWS_PROFILE=None

# You acan also set these 4 using env variables or allowing the code to read your ~/.aws_credentials file - (if file, set AWS_PROFILE)
CLOUDFLARE_SECRET_ACCESS_KEY = ''
CLOUDFLARE_ACCESS_KEY_ID = ''

# Our version of the parquet files is available in the kamiwaza public bucket
CLOUDFLARE_ENDPOINT = 'https://55cce94494ffc59c1fd1bb8ce797f369.r2.cloudflarestorage.com'
CLOUDFLARE_REGION = 'enam'

In [ ]:
# administrivia - set up logger, suppress debug - this version assumes the secrets exist!
# (if they don't, see wiki-s3-ingest.ipynb for an example of getting them set
import logging
logging.basicConfig(level=logging.WARNING)
import os
import time
logger = logging.getLogger()

# Check for environment variables
CLOUDFLARE_SECRET_ACCESS_KEY = os.getenv('CLOUDFLARE_SECRET_ACCESS_KEY') if not CLOUDFLARE_SECRET_ACCESS_KEY else CLOUDFLARE_SECRET_ACCESS_KEY
CLOUDFLARE_ACCESS_KEY_ID = os.getenv('CLOUDFLARE_ACCESS_KEY_ID') if not CLOUDFLARE_ACCESS_KEY_ID else CLOUDFLARE_ACCESS_KEY_ID
CLOUDFLARE_ENDPOINT = os.getenv('CLOUDFLARE_ENDPOINT') if not CLOUDFLARE_ENDPOINT else CLOUDFLARE_ENDPOINT
CLOUDFLARE_REGION = os.getenv('CLOUDFLARE_REGION') if not CLOUDFLARE_REGION else CLOUDFLARE_REGION

# If environment variables are not set, look in ~/.aws/credentials;
if not CLOUDFLARE_SECRET_ACCESS_KEY or not CLOUDFLARE_ACCESS_KEY_ID:
    if AWS_PROFILE:
        config = ConfigParser()
        config.read(os.path.expanduser("~/.aws/credentials"))
        if AWS_PROFILE in config.sections():
            CLOUDFLARE_SECRET_ACCESS_KEY = config.get(AWS_PROFILE, 'aws_secret_access_key')
            CLOUDFLARE_ACCESS_KEY_ID = config.get(AWS_PROFILE, 'aws_access_key_id')



# THIS CELL IS MARKDOWN
```python
# Note that in our example here we have two options: setting
# your own credentials up, or setting just the endpoint/region and using
# Kamiwaza's

# In real use in Kamiwaza when you ingest, Kamiwaza maps
# Catalog Object -> Name of Credential (catalog metadata) -> Actual Secret

# So for example
# Catalog entry -> 'cloudflare_secret_access_key' (metadata for secret_access_key) -> [The Actual Secret, stored as the 'cloudflare_secret_access_key' secret]
# Which means you can rotate credentials without rotating them on a per entry basis
# 
# BUT once a secret EXISTS (eg, cloudflare_secret_access_key as below) you can actually pass None to the ingester
# And if you have access to the secret it will look it up and use it and also store it as metadata; allowing you to 
# associate new catalog entries with existing secrets. That's shown below; but this cell won't work for you
# without changes, but that's a key point!


from kamiwaza.services.retrieval.services import RetrievalService
from kamiwaza.services.catalog.ingest.s3 import S3Ingester

# In this example (as opposed to in @wiki-s3-ingest.ipynb) we say None because
# we are not providing credentials, just referring to existing secrets
s3i = S3Ingester(
    disable_ssl_verification=True,
    secret_access_key = {'cloudflare_secret_access_key': None},
    access_key_id = {'cloudflare_access_key_id': None},
    endpoint_url = {'cloudflare_endpoint': None},
    region = {'cloudflare_region': None}
)

s3i.ingest(bucket='kamiwaza-demo', path='sampleparquet/')
```





In [ ]:
# SETTING UP CREDS
# Don't need to do this if these exist
# in this case we are setting them assuming the catalog
# does not have these secrets (cloudflare_secret_access_key, etc) in it


import logging
logging.basicConfig(level=logging.WARNING)
import os
import time
from configparser import ConfigParser


# Check for environment variables if they were not staticially set; if these are not set
#   we will next check our ~/.aws/credentials file
# (For the record, this is how we typically use this)
if not CLOUDFLARE_SECRET_ACCESS_KEY:
    CLOUDFLARE_SECRET_ACCESS_KEY = os.getenv('CLOUDFLARE_SECRET_ACCESS_KEY', None)
if not CLOUDFLARE_ACCESS_KEY_ID:
    CLOUDFLARE_ACCESS_KEY_ID = os.getenv('CLOUDFLARE_ACCESS_KEY_ID', None)

# If environment variables are not set, look in ~/.aws/credentials
if not CLOUDFLARE_SECRET_ACCESS_KEY or not CLOUDFLARE_ACCESS_KEY_ID:
    config = ConfigParser()
    config.read(os.path.expanduser("~/.aws/credentials"))
    if AWS_PROFILE in config.sections():
        CLOUDFLARE_SECRET_ACCESS_KEY = config.get(AWS_PROFILE, 'aws_secret_access_key')
        CLOUDFLARE_ACCESS_KEY_ID = config.get(AWS_PROFILE, 'aws_access_key_id')

# Public bucket, so we don't need these!
#if not CLOUDFLARE_SECRET_ACCESS_KEY or not CLOUDFLARE_ACCESS_KEY_ID:
#    raise ValueError("Cloudflare keys not found in environment variables or ~/.aws/credentials")

cloudflare_endpoint = CLOUDFLARE_ENDPOINT
cloudflare_region = CLOUDFLARE_REGION

In [ ]:
from kamiwaza.services.catalog.ingest.s3 import S3Ingester

# Example of passing credentials to be used and stored in the catalog for retrieval in the future
# This usage will not overwrite if they exist - so if you have different credentials
# you will need to delete them! (you can also call create_secret() in the catalog plugin;
# and eventually delete_secret() which is not in yet!)
s3i = S3Ingester(
    disable_ssl_verification=True,
    secret_access_key = {'cloudflare_secret_access_key': CLOUDFLARE_SECRET_ACCESS_KEY},
    access_key_id = {'cloudflare_access_key_id': CLOUDFLARE_ACCESS_KEY_ID},
    endpoint_url = {'cloudflare_endpoint': cloudflare_endpoint},
    region = {'cloudflare_region': cloudflare_region}
)

s3i.ingest(bucket=BUCKET_NAME, path=BUCKET_PATH)

# let catalog get consistent
time.sleep(3)


In [ ]:
# catalog.get_datasets is currently a full text match eg *testdata*
# we will support exact matching in the future
list(s3i.catalog.get_datasets('testdata'))

Expected Output:

```
['urn:li:dataset:(urn:li:dataPlatform:s3,testdata/customers.parquet,PROD)',
 'urn:li:dataset:(urn:li:dataPlatform:s3,testdata/order_items/date=2024-01-01/data.parquet,PROD)',
 'urn:li:dataset:(urn:li:dataPlatform:s3,testdata/order_items/date=2024-01-02/data.parquet,PROD)',
 'urn:li:dataset:(urn:li:dataPlatform:s3,testdata/order_items/date=2024-01-03/data.parquet,PROD)',
 'urn:li:dataset:(urn:li:dataPlatform:s3,testdata/orders/date=2024-01-01/data.parquet,PROD)',
 'urn:li:dataset:(urn:li:dataPlatform:s3,testdata/orders/date=2024-01-02/data.parquet,PROD)',
 'urn:li:dataset:(urn:li:dataPlatform:s3,testdata/orders/date=2024-01-03/data.parquet,PROD)']
```

In [ ]:
# in this cell, we get the contents as a ray dataset; retrieve datasets
# calls a ray.data.* loader function and returns the actual ray dataset objects, see
# https://docs.ray.io/en/master/data/api/dataset.html
# Ray can actually read parquet directly (and do things like shard shuffle it)
# But in our demo we actually take each dataset as a row, we get the full
# bytes of the file, and read those directly into a pandas data frame
# and head() each file; Kamiwaza's default ray.data retrieval is read_binary_data
# but you should override that unless:
# 1. You need the bytes specifically in order
# 2. You are actually reading binary data
#
# read_text and read_parquet are useful and we will demostrate elsewhere!

import pandas as pd
import io
from kamiwaza.services.retrieval.services import RetrievalService

# This is lazily loaded, so nothing is retrieved until something requires it to materialize
ds = RetrievalService().retrieve_datasets(query='testdata')

# in this case, each 'row' is a file with bytes
for dataset in ds.iter_rows():
    data = io.BytesIO(dataset['bytes'])
    df = pd.read_parquet(data)
    print(df.head())
    print("\n\n")